In [15]:
from dataclasses import dataclass
import numpy as np

In [3]:
@dataclass
class XYZ:
    x: int
    y: int
    z: int

In [9]:
def parse_XYZ(line):
    parts = [int(x.strip()) for x in line.strip().split(', ')]
    return XYZ(*parts)

In [10]:
@dataclass
class Ray:
    start: XYZ
    velocity: XYZ

def parse_Ray(line):
    parts = line.strip().split(' @ ')
    return Ray(start=parse_XYZ(parts[0]), velocity=parse_XYZ(parts[1]))

In [11]:
with open('input', 'r') as f:
    input_file = f.readlines()

In [12]:
rays = [parse_Ray(line) for line in input_file]

In [ ]:
# 0: s.x, 1: s.y, 2: s.z, 3: s.v.x, 4: s.v.y, 5: s.v.z, 6: t_0, .... 

equations = []

for ray in rays:
    pass